import training and testing datasets

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import itertools as it
import pickle_utils as pu
import matplotlib.pyplot as plt

In [ ]:
TRAIN_PATH = './us_census_full/census_income_learn.csv'
TEST_PATH = './us_census_full/census_income_test.csv'

# set hard coded information
con_i = [0,5,16,17,18,30,39]    # indices of continous features
non_i = list(set(np.arange(42))-set(con_i)) # indices of non-continous (discrete) features
dtype = dict(it.chain(
      zip(con_i, it.repeat(np.float32)),
      zip(non_i, it.repeat('category'))))
headers = ["age","class of worker","detailed industry recode",
         "detailed occupation recode","education",
         "wage per hour","enroll in edu inst last wk",
         "major industry code","major occupation code","marital stat","race",
         "hispanic origin","sex","member of a labor union","reason for unemployment",
         "full or part time employment stat","capital gains",
         "capital losses","dividends from stocks","tax filer stat",
         "region of previous residence","state of previous residence",
         "detailed household and family stat","detailed household summary in household",
         "instance weight","migration code-change in msa",
         "migration code-change in reg","migration code-move within reg",
         "live in this house 1 year ago","migration prev res in sunbelt",
         "num persons worked for employer","family members under 18",
         "country of birth father",
         "country of birth mother","country of birth self",
         "citizenship","own business or self employed",
         "fill inc questionnaire for veteran's admin",
         "veterans benefits","weeks worked in year","year","label"]

# read data
# note: the missing value is represented by " ?"
df_train = pd.read_csv(TRAIN_PATH, dtype=dtype, header=None,
               names=headers, na_values=" ?")
df_test = pd.read_csv(TEST_PATH, dtype=dtype, header=None,
               names=headers, na_values=" ?")
df = pd.concat([df_train,df_test])

# make sure all the discrete features are of type "category"
for i in non_i:
    df.iloc[:,i] = df.iloc[:,i].astype("category")

determine the position of the feature "instance weight" by comparing the result of the following function and the "distinct values" provided in the file "census_income_metadata.txt". The reason for doing this is that this feature should not be used when training/testing classifiers. 

In [ ]:
# this function takes a dataframe as input and output:
#     dis_sizes (a list): number of distinct values of each feature
#     dis_values (a list of list): distinct values of each feature
def numberDistinctValues(df):
    dis_values = []
    dis_sizes = []
    for i in range(df.shape[1]):
        temp = list(df.iloc[:,i].unique())
        dis_values.append(temp)
        dis_sizes.append(len(temp))
    return dis_sizes, dis_values

dis_sizes, _ = numberDistinctValues(df_train)
print(dis_sizes)

It is easy to see that the "instance weight" is the 25th feature in the dataframes.

Then, we delete duplicated samples (rows) in the dataframe:

In [ ]:
df = df.drop_duplicates()

Task 1: statistic based and univariate audit of the dataset

Firstly, we plot histograms to show distribution of both numerical and categorical features.
Warning: it takes a relative long time to output all the plots.

In [ ]:
def getCategoricalIndices(df):
    output = []
    for i in range(df.shape[1]-1):
        if(df.dtypes[i].name=='category'):
            output.append(i)
    return output

In [ ]:
def plotContFeature(df, i):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlabel("bins")
    ax.set_ylabel("Frequency")
    ax.set_title("distribution of feature: {:s} (numerical)".format(df.columns.values[i]))
    df.iloc[:,i].plot.hist()

def plotCateFeature(df, i):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylabel("Frequency")
    ax.set_title("distribution of feature: {:s} (categorical)".format(df.columns.values[i]))
    df.iloc[:,i].value_counts().plot(kind='bar')

for i in con_i:
    plotContFeature(df, i)
    
for i in getCategoricalIndices(df):
    plotCateFeature(df, i)

Then, we plot boxplots for numerical features to show the quantiles and extreme values

In [ ]:
def boxplotContFeatures(df, i):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.boxplot(df.iloc[:,i], showfliers=False)
    ax.set_title("box plot of feature {} (numerical)".format(df.columns.values[i]))
    ax.scatter([1, 1], [df.iloc[:,i].min(), df.iloc[:,i].max()])
print("In the following boxplots, extreme values are shown as blue dots, 25% and 75% \
quantile are shown as a rectangle, the median value is shown as a orange line")
for i in con_i:
    boxplotContFeatures(df, i)

We can use "describe" function to obtain the above information along with other information

In [ ]:
num_descript = df.describe(include=[np.number]) # gives mean, standard deviation, mean,
                                 # min, 25%, 50%, 75% percentile, max,
cat_descript = df.describe(include=['category']) # gives number of distinct values,
                                  # most frequent category and it frequency

Finally, we print the missing rates of the features:

In [ ]:
# return missing rate of a dataframe
def missing_rate(df):
    return np.sum(pd.isnull(df).values,axis=0)/df.shape[0]

print("only features with missing values are displayed")
print("numerical features' missing rate:")
missing_num = missing_rate(df.iloc[:, con_i])
for i in range(len(con_i)):
    if(missing_num[i]!=0):
        print("feature #{}, {} : {:.3f}".format(con_i[i],
            df.columns.values[con_i[i]], missing_num[i]))

print("categorical features' missing rate:")
missing_cat = missing_rate(df.iloc[:,non_i])
for i in range(len(non_i)):
    if(missing_cat[i]!=0):
        print("feature #{}, {} : {:.3f}".format(non_i[i],
            df.columns.values[non_i[i]], missing_cat[i]))

Task 2 data preprocssing and model selection

For the features that have high missing rates (>= 40%), we simply eliminate them. Then, we  ignore samples that have missing features:

In [ ]:
high_missing_indices = np.nonzero(missing_rate(df)>0.4)[0]
# eliminate features with high missing rate and the feature "istance_weight"
new_indices1 = list(set(range(len(headers)))-set(list(high_missing_indices)+[24]))
df1 = df.iloc[:,new_indices1]
# eliminate samples with at least one missing value
df2 = df1.dropna()

We firstly choose to test Naive Bayes Classifier (NBC) for the following reasons:
    1. It can handle categorical features and numerical features at the same time.
    2. It can handle categorical features with many levels in a way that does not depend on how we encode the categories. This advantage is particular important in our case, since we have some variables that have many categories.

Note: since there is no available NBC implementation for mixed dataset (categorical & numerical), I implement it with a similar interface with other classifiers in sklearn. Please see the file "naive_bayes_classifier.py"

Firstly, let us introduce some useful functions.

In [ ]:
# encode the categorical features
def encodeCateFeatures(df):
    df_encoded = df.copy()
    for i in getCategoricalIndices(df):
        df_encoded.iloc[:,i] = df.iloc[:,i].cat.codes.astype("category")
    return df_encoded

# train-test set split
def split_dataset(df, ratio):
    N = df.shape[0]
    N_train = int(ratio*N)
    data = df.values[:,:-1].astype(np.float64)
    label = df.values[:,-1]
    return ((data[:N_train,:],label[:N_train]),
              (data[N_train:,:],label[N_train:]))

# train a model on training set and return its mean accuracy on validation/test set 
def evaluateModel(X_train, y_train, X_test, y_test, estimator):
    estimator.fit(X_train, y_train)
    return estimator.score(X_test, y_test)

# 5-fold cross validation on a set for model selection
from sklearn.model_selection import StratifiedKFold
def CV_Selection(X, y, estimator, n_folds=5):
    cv = StratifiedKFold(n_splits=n_folds)
    scores = []
    for train_index, valid_index in cv.split(X, y):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        s = evaluateModel(X_train, y_train, X_valid, y_valid, estimator)
        scores.append(s)
    mean_score = np.mean(scores)
    print("The {}-fold cross validated accuracy is {:.4f}".format(n_folds,
                                                            mean_score))
    return mean_score

Evaluation of NBC model:

In [ ]:
def getCategoricalValues(df):
    cate_indices = getCategoricalIndices(df)
    values = []
    _, dis_values = numberDistinctValues(df.iloc[:,cate_indices])
    return dis_values

# encode categorical features with integers
df3 = encodeCateFeatures(df2)
# split the dataset into a set for training and validation, and a set for testing
(X_train, y_train),(X_test, y_test) = split_dataset(df3, 2/3)
# build NBC classifier
from naive_bayes_classifier import NaiveBayesClassifier
new_cate_indices = getCategoricalIndices(df2)
cate_classes = getCategoricalValues(df3)
nbc = NaiveBayesClassifier(df3.shape[1]-1, new_cate_indices, cate_classes)
# perform stratified 5-fold cross validation on the training validation set
CV_Selection(X_train, y_train, nbc, n_folds=5)

Result: mean accuracy of 5-fold CV on training/validation set of NBC is 0.8178

This result is very bad considering that simply predicting all the samples being of class "under 50000" can achieve accuracy of 0.9367. This bad result is due to that NBC's condiontional indpendence assumption is too strong. In other words, the correlation between features is important for the prediction. As a result, the next model we test is logistic regression. 

For logistic regression, how we enocde categorical features has potentially a large effect on the prediction performance. One hot encoding is a reasonable approach, but it will result in a prohibitive number of features in our case. This difficulty can be easily solved if we group together categories that are very little represented (for example, categories that have less than 0.5% of samples).

Evaluation of Logistic Regression model:

In [ ]:
# grouping categories that have fraction of samples under a specified threshold
# The grouped category is called "Rare Cases"
def groupRareCateCases(df, threshold=0.002):
    df_grouped = df.copy()
    for i in getCategoricalIndices(df):
        temp = df.iloc[:,i].value_counts()
        to_replace = temp.loc[temp<threshold*df.shape[0]].index
        if(to_replace.size>0):
            df_grouped.iloc[:,i] = df.iloc[:,i].replace(to_replace,
                  ' Rare cases').astype("category")
    return df_grouped

# reorder dataframe to make sure the "label" is at the last column
def putLabelToLastColumn(df):
    cols = df.columns.tolist()
    i = 0
    while(True):
        if(cols[i]=="label"):
            break
        i += 1
    cols_reordered = cols[:i] + cols[(i+1):] + [cols[i]]
    return df[cols_reordered]

df2_grouped = groupRareCateCases(df2, 0.002)
cols = df2_grouped.columns.values[getCategoricalIndices(df2_grouped)]
df3_onehot = pd.get_dummies(df2_grouped, columns=cols, drop_first=True)
df3_onehot = putLabelToLastColumn(df3_onehot)

(X_train, y_train),(X_test, y_test) = split_dataset(df3_onehot, 2/3)
from sklearn.linear_model import LogisticRegression as LR
lr = LR()
CV_Selection(X_train, y_train, lr, n_folds=5)

Result: mean accuracy of 5-fold CV on training/validation set of Logistic Regression (categories under 0.2% representation rate are grouped together) is 0.9525

We also show the results with other parameters:

Not one-hot encoding (40 features):                                      0.9457

One-hot encoding + Categories grouping (threshold=5%, 72 features):      0.9488

One-hot encoding + Categories grouping (threshold=4%, 83 features):      0.9496

One-hot encoding + Categories grouping (threshold=3%, 98 features):      0.9500

One-hot encoding + Categories grouping (threshold=2%, 120 features):     0.9512

One-hot encoding + Categories grouping (threshold=1%, 225 features):     0.9521

One-hot encoding + Categories grouping (threshold=0.5%, 314 features):   0.9523

One-hot encoding + Categories grouping (threshold=0.2%, 368 features):   0.9525

One-hot encoding + Categories grouping (threshold=0.1%, 434 features):   0.9525

Finally, we evaluate the best model on the test set. The best model (according to cross-validation on traning/validation set) is using one-hot encoding and categories grouping (threshold=0.2%).

In [ ]:
evaluateModel(X_train, y_train, X_test, y_test, lr)

Result: the mean accuracy on the test set of the selected model is 0.9526

In order to obtain a clear insight on the most predictive features for this classification problem, we perform feature selection using L1 penalty.

In [ ]:
# when performing feature selection, we don't use one-hot encoding
df3 = encodeCateFeatures(df2)
(X_train, y_train),(X_test, y_test) = split_dataset(df3, 2/3)
lr_l1 = LR(C=0.0001, penalty="l1").fit(X_train, y_train)
coeff_l1 = np.squeeze(lr_l1.coef_)
selected_features = np.nonzero(coeff_l1!=0)[0]
df_base = df3
print(df_base.columns[selected_features])

We test the feature selection using the best model that we selected previously.

In [ ]:
# select features
df2_selected = pd.concat([df3.iloc[:,selected_features], 
                          df3.iloc[:,-1]], axis=1)
# grouping rare categories and using one-hot encoding
df2_selected_grouped = groupRareCateCases(df2_selected, 0.005)
cols = df2_selected_grouped.columns.values[getCategoricalIndices(df2_selected_grouped)]
df3_selected_onehot = pd.get_dummies(df2_selected_grouped, columns=cols, drop_first=True)
df3_selected_onehot = putLabelToLastColumn(df3_selected_onehot)
# train-test set split
(X_train, y_train),(X_test, y_test) = split_dataset(df3_selected_onehot, 2/3)
# train model on training set and evaluate it on test set
from sklearn.linear_model import LogisticRegression as LR
lr = LR()
evaluateModel(X_train, y_train, X_test, y_test, lr)

Result:

When C=10^(-4) (inverse regularization strength), the mean accuracy is 0.9467, the selected features are: 'age', 'detailed occupation recode', 'wage per hour', 'major occupation code', 'capital gains', 'capital losses', 'dividends from stocks', 'tax filer stat', 'state of previous residence', 'detailed household and family stat', 'country of birth father', 'country of birth mother', 'country of birth self', 'weeks worked in year'

When C=10^(-5), the mean accuracy is 0.9439, the selected features are:
'wage per hour', 'capital gains', 'capital losses', 'dividends from stocks', 'state of previous residence', 'country of birth self', 'weeks worked in year'.

When C=5*10^(-7), the mean accuracy is 0.9427, the selected features are: 'wage per hour', 'capital gains', 'dividends from stocks', 'country of birth self'

When C=2*10^(-7), the mean accuracy is 0.9426, the selected features are: 'wage per hour', 'capital gains'

When C=10^(-7), the mean accuracy is 0.9422, the selected features are: 'capital gains'

In conclusion, the most predictive features are 'capital gains' and 'wage per hour'.